In [1]:
import sys
import os, os.path

sys.path.append(os.path.join(os.getcwd() ,'/modules'))
root_path = "C:/git/Springboard-Public/Capstone Project 2/"
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = "/content/drive/My Drive/Capstone Project 2/"

print('Current Working Dir: ', os.getcwd())
print('Root Path: ', root_path)

# We need to set the working directory since we are using relative paths from various locations
if os.getcwd() != root_path:
  os.chdir(root_path)

Current Working Dir:  C:\git\Springboard-Public\Capstone Project 2\notebooks
Root Path:  C:/git/Springboard-Public/Capstone Project 2/


In [2]:
from modules.lib.StandardTraining import *
from modules.models.ResNet import ResNet_GrayScale_12_Out

from torchsummary import summary

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

%matplotlib inline

In [3]:
force_cpu = True
device = torch.device('cuda' if ~force_cpu and torch.cuda.is_available() else 'cpu')
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(f'Working on device={device}')

Working on device=cuda


In [4]:
net = ResNet_GrayScale_12_Out(layers=18, drop_out_precent=.5) #18, 34, 50, 101, 152

net = nn.DataParallel(net)
net.to(device)


summary(net, (1, 320, 320))   

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 160, 160]           3,136
       BatchNorm2d-2         [-1, 64, 160, 160]             128
              ReLU-3         [-1, 64, 160, 160]               0
         MaxPool2d-4           [-1, 64, 80, 80]               0
            Conv2d-5           [-1, 64, 80, 80]          36,864
       BatchNorm2d-6           [-1, 64, 80, 80]             128
              ReLU-7           [-1, 64, 80, 80]               0
            Conv2d-8           [-1, 64, 80, 80]          36,864
       BatchNorm2d-9           [-1, 64, 80, 80]             128
             ReLU-10           [-1, 64, 80, 80]               0
       BasicBlock-11           [-1, 64, 80, 80]               0
           Conv2d-12           [-1, 64, 80, 80]          36,864
      BatchNorm2d-13           [-1, 64, 80, 80]             128
             ReLU-14           [-1, 64,

In [5]:
standardTraining = StandardTraining(number_images=0, 
                                     batch_size=32, 
                                     learning_rate=1e-5, 
                                     num_epochs=20,
                                     device=device, 
                                     net=net,
                                     affineDegrees=8, 
                                     translatePrecent=0.8, 
                                     shearDegrees=8, 
                                     brightnessJitter=0.3, 
                                     contrastJitter=0.2, 
                                     augPercent=0.3,
                                     use_positivity_weights=True,
                                     observation_min_count = 100,
                                     l2_reg=1e-4)

Total Oversampled Rows Added to Train: 48,764

Number of Training Images: 154,080
Number of Validation Images: 26,432

Positive Weights used in BCEWithLogitsLoss:


Enlarged_Cardiomediastinum     6.675600
Cardiomegaly                   4.219335
Lung_Opacity                   1.072221
Lung_Lesion                    7.242658
Edema                          2.749817
Consolidation                  6.120477
Pneumonia                      9.754519
Atelectasis                    3.636634
Pneumothorax                   6.247072
Pleural_Effusion               1.628320
Pleural_Other                 12.927506
Fracture                       7.460820
dtype: float64

In [6]:
standardTraining.train()

▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [1/20]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 4964.2248         
Time of Completion: 2020-07-28 10:26:28.710107          
Training Time: 0:27:22.722822          
Validation Time: 0:03:54.122403

TRAINING

▔▔▔▔▔


,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,29521,0.589817,0.254164,0.355245,0.639077,0.317706
4,Edema,41090,0.670650,0.379254,0.484514,0.694221,0.434152
5,Consolidation,21639,0.608485,0.194657,0.294956,0.650383,0.267397
7,Atelectasis,33231,0.558424,0.242559,0.338212,0.559863,0.255016
9,Pleural_Effusion,58623,0.617590,0.480663,0.540591,0.652680,0.522993



VALIDATION

▔▔▔▔▔▔▔


,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,3002,0.457029,0.206254,0.284235,0.686097,0.202982
4,Edema,6172,0.668989,0.401576,0.501884,0.751369,0.440621
5,Consolidation,1582,0.155499,0.112175,0.130331,0.646160,0.097942
7,Atelectasis,4162,0.369294,0.190932,0.251720,0.582999,0.190370
9,Pleural_Effusion,8877,0.546018,0.550170,0.548086,0.734311,0.553070


▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [2/20]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔


KeyboardInterrupt: 

In [ ]:
standardTraining.displayMetrics()